# TRAINING 
---
## IMPORT 

In [26]:
import pickle
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.optimizers import Adam
from sklearn.feature_extraction.text import CountVectorizer

from tensorflow.python.keras import models
from tensorflow.python.keras.layers import Dense
from tensorflow import keras
from tensorflow.keras.callbacks import TensorBoard

import os

## Load Data

In [27]:
with open('../data/x_train.pickle', 'rb') as handle:
    X_train = pickle.load(handle)
with open('../data/y_train.pickle', 'rb') as handle:
    y_train = pickle.load(handle)
with open('../data/x_val.pickle', 'rb') as handle:
    X_val = pickle.load(handle)
with open('../data/y_val.pickle', 'rb') as handle:
    y_val = pickle.load(handle)

In [28]:
X_train.head()

,id,question,question_body,answer
490144,21032057,Deleting multiple indexes list python,My problem I list eg lst lst pop lst pop Becau...,You use list comprehension rebuild list indice...
815071,34022443,Django Grappelli Nested Inlines create new nes...,I looking way create new nested row saving own...,What take templates django grappelli inline pr...
62146,2185875,Expanding elements list,I 'm looking nice way process list elements ne...,The last one probably pythonic could try impli...
304024,12626260,UnicodeDecodeError processing filenames,I 'm using Python Ubuntu x I files folder file...,Your filenames byte strings contain UTF bytes ...
187856,7288407,Python string manipulation performance problems,I following piece code I execute around millio...,EDIT I 'm changing answer bit I 'll leave orig...


In [29]:
# Mean length of body x and y
avg_question = sum(len(word) for word in X_train['question']) / len(X_train['question'])
avg_question_body = sum(len(word) for word in X_train['question_body']) / len(X_train['question_body'])
avg_answer = sum(len(word) for word in X_train['answer']) / len(X_train['answer'])
print(avg_question)
print(avg_question_body)
print(avg_answer)

40.827
718.1521
414.0572


## Preprocessing

In [30]:
def vectorize_data(X_train, X_val, max_features=1000):    
    vectorizer = CountVectorizer(max_features=max_features)

#     X_train_vect = X_train
#     X_test_vect = X_test

#     X_train_vect['Title'] = vectorizer.fit_transform(X_train_vect['Title'])
#     X_test_vect['Title'] = vectorizer.transform(X_test_vect['Title'])

#     X_train_vect['Body_x'] = vectorizer.fit_transform(X_train_vect['Body_x'])
#     X_test_vect['Body_x'] = vectorizer.transform(X_test_vect['Body_x'])

#     X_train_vect['Body_y'] = vectorizer.fit_transform(X_train_vect['Body_y'])
#     X_test_vect['Body_y'] = vectorizer.transform(X_test_vect['Body_y'])

    X_train_vect = vectorizer.fit_transform(X_train)
    X_val_vect = vectorizer.transform(X_val)

    return X_train_vect, X_val_vect

In [31]:
X_train_vect, X_test_vect = vectorize_data(X_train['answer'], X_val['answer'], max_features=1000)

In [32]:
# Même longeur de texte
# maxlenBodyX = 750
# maxlenBodyY = 750
# maxLenTitle = 60

# x_train_pad = X_train_vect
# x_test_pad = X_test_vect

# x_train_pad['Title'] = sequence.pad_sequences(x_train_pad['Title'], maxLenTitle)
# x_train_pad['Body_x'] = sequence.pad_sequences(x_train_pad['Body_x'], maxlenBodyX)
# x_train_pad['Body_y'] = sequence.pad_sequences(x_train_pad['Body_y'], maxlenBodyY)

# x_test_pad['Title'] = sequence.pad_sequences(x_test_pad['Title'], maxLenTitle)
# x_test_pad['Body_x'] = sequence.pad_sequences(x_test_pad['Body_x'], maxlenBodyX)
# x_test_pad['Body_y'] = sequence.pad_sequences(x_test_pad['Body_y'], maxlenBodyY)


## Create model

In [33]:
def build_model(input_shape, DROPOUT_RATE = 0.2, UNITS = 64, NUM_CLASSES = 2):

    model = keras.Sequential()
    model.add(Dense(units=50, activation='relu'))
    model.add(Dense(units=1, activation='sigmoid'))
    model.build(input_shape)
    model.summary()
    return model

In [34]:
X_train_vect[0].shape[1]

1000

In [35]:
model = build_model(X_train_vect.shape)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (10000, 50)               50050     
_________________________________________________________________
dense_3 (Dense)              (10000, 1)                51        
Total params: 50,101
Trainable params: 50,101
Non-trainable params: 0
_________________________________________________________________


## Compilation

In [36]:
INIT_LR = 0.001
EPOCHS = 10

In [37]:
optimizer = Adam(learning_rate=INIT_LR, decay=INIT_LR/EPOCHS)

In [38]:
model.compile(
  optimizer = optimizer,
  loss = "binary_crossentropy",
  metrics = ["accuracy"]
)

## Entrainement

In [39]:
def train_model(X_train, y_train, EPOCHS = 100, BATCH_SIZE = 128, patience=5):
    # For tensorboard
#     os.makedirs("./logs",exist_ok=True)
#     tensorboard = TensorBoard(log_dir=os.path.join('./logs'), histogram_freq=0,
#                                 write_graph=True, write_images=False,profile_batch = 100000000)

#     callbacks = [tensorboard]

    history = model.fit(X_train, y_train, epochs=EPOCHS, batch_size=BATCH_SIZE) # , callbacks=callbacks
    return history

In [40]:
history = train_model(X_train_vect, y_train, EPOCHS = 10, BATCH_SIZE = 128, patience=5)

Epoch 1/10


InvalidArgumentError:  TypeError: 'SparseTensor' object is not subscriptable
Traceback (most recent call last):

  File "/usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/script_ops.py", line 247, in __call__
    return func(device, token, args)

  File "/usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/script_ops.py", line 135, in __call__
    ret = self._func(*args)

  File "/usr/local/lib/python3.6/dist-packages/tensorflow/python/autograph/impl/api.py", line 620, in wrapper
    return func(*args, **kwargs)

  File "/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/data_adapter.py", line 504, in py_method
    return [slice_array(inp) for inp in flat_inputs]

  File "/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/data_adapter.py", line 504, in <listcomp>
    return [slice_array(inp) for inp in flat_inputs]

  File "/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/data_adapter.py", line 503, in slice_

## Analyse des résultats

In [ ]:
y_test_pred = model.predict_classes(X_test_vect)

In [ ]:
from sklearn.metrics import confusion_matrix

cf_matrix = confusion_matrix(y_test, y_test_pred)
print(cf_matrix)

In [ ]:
import seaborn as sns
sns.heatmap(cf_matrix/np.sum(cf_matrix), annot=True, fmt='.2%', cmap='Blues')

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=e80043e2-6875-4b65-a196-a0ffb97a1282' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>